# Vue Dapp GG20 Donations

- From: 2024/04/23 12:00 UTC	
- To: 2024/05/07 23:59 UTC
- Arbiscan API: https://docs.arbiscan.io/api-endpoints/accounts#get-a-list-of-internal-transactions-by-address
- Contract Internal Transactions https://arbiscan.io/txsInternal?a=0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187
- Token Transfers https://arbiscan.io/tokentxns?a=0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187

In [1]:
from web3 import Web3
import pandas as pd
import json
import requests
from decimal import Decimal
import os
from dotenv import load_dotenv
load_dotenv()
ARBISCAN_API_KEY = os.getenv('ARBISCAN_API_KEY')


target_address = Web3.to_checksum_address("0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187")

url = (
	"https://api.arbiscan.io/api"
	"?module=account"
	f"&action=txlistinternal" # action
	f"&address={target_address}"
	f"&startblock=0"
	f"&sort=desc"
	f"&apikey={ARBISCAN_API_KEY}"
)

txs = json.loads(requests.get(url).text)['result']

def get_df_by_txs(txs):
	transactions = []
	for tx in txs:
		data = {
			'hash': tx['hash'],
			'timeStamp': tx['timeStamp'],
			'from': Web3.to_checksum_address(tx['from']),
			'to': Web3.to_checksum_address(tx['to']),
			'value': int(tx['value'])
		}
		transactions.append(data)
	
	df = pd.DataFrame(transactions)
	df['timestamp'] = pd.to_datetime(df['timeStamp'].astype(int), unit='s')
	del df['timeStamp']
	
	df['hourly_timestamp'] = pd.to_datetime((df['timestamp'].apply(lambda x: x.timestamp() / 3600)).astype(int), unit='h')

	return df

def filter_gg20_time_range(df):
	return df[(df['timestamp'] >= '2024-04-23 12:00:00') & (df['timestamp'] <= '2024-05-07 23:59:59')]

df_eth_donations = get_df_by_txs(txs)
df_eth_donations = filter_gg20_time_range(df_eth_donations)
df_eth_donations['token'] = 'ETH'
df_eth_donations



# only show "to" is the target address and "from" is not the target address
# df_eth_donations = df_eth_donations[df_eth_donations['to'] == target_address]
# df_eth_donations = df_eth_donations[df_eth_donations['from'] != target_address]

,hash,from,to,value,timestamp,hourly_timestamp,token
0,0x31623d265c4a96468e8e3f52b1779d39fbb455d125d8...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1000000000000000,2024-05-07 23:39:51,2024-05-07 23:00:00,ETH
1,0x414efe0670ed36b8026bd9633236e06d3c41247488b8...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,400000000000000,2024-05-07 23:16:05,2024-05-07 23:00:00,ETH
2,0x4f69f00ce3ef0130d880e9b7c2df7009c47835bc3b36...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,450000000000000,2024-05-07 22:54:26,2024-05-07 22:00:00,ETH
3,0xc757eebeff0bd14c0d9fb49049794c0fd0f662bae924...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,330000000000000,2024-05-07 22:52:04,2024-05-07 22:00:00,ETH
4,0xe0c7961e7b9d459ab7d1d4fde33bef6c1a0782ebca6d...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,340000000000000,2024-05-07 22:49:29,2024-05-07 22:00:00,ETH
...,...,...,...,...,...,...,...
478,0xb7dbe3026ee6cd798eed75446954ad946407d16da76f...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,5310000000000,2024-04-25 03:29:57,2024-04-25 03:00:00,ETH
479,0xb8cc8ef2cb01e0b9fed6e1ba469c57eddb52d151b76f...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,10000000000000,2024-04-25 03:24:26,2024-04-25 03:00:00,ETH
480,0x22dd457c866498fd386fe1eec918082e8da495706a8f...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,10000000000000,2024-04-25 03:24:24,2024-04-25 03:00:00,ETH
481,0xe0173106099473a467b22eb2aeece57557ecb412997c...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,600000000000000,2024-04-24 17:37:28,2024-04-24 17:00:00,ETH


In [2]:
# ETH usd_price

with open('gg20/eth_gg20_historical_price.json') as f:
	eth_gg20_historical_price = json.load(f)

# convert to dataframe
df_eth_gg20_historical_price = pd.DataFrame(eth_gg20_historical_price)
df_eth_gg20_historical_price['timestamp'] = pd.to_datetime(df_eth_gg20_historical_price['timestamp'], unit='ms')
df_eth_gg20_historical_price['hourly_timestamp'] = pd.to_datetime(df_eth_gg20_historical_price['timestamp'].apply(lambda x: x.timestamp() / 3600).astype(int), unit='h')

# rename price with usd_price
df_eth_gg20_historical_price = df_eth_gg20_historical_price.rename(columns={'price': 'usd_price'})

df_eth_gg20_historical_price

,timestamp,usd_price,hourly_timestamp
0,2024-04-23 12:03:45.518,3179.641560,2024-04-23 12:00:00
1,2024-04-23 13:02:52.599,3187.390157,2024-04-23 13:00:00
2,2024-04-23 14:00:10.173,3205.414595,2024-04-23 14:00:00
3,2024-04-23 15:02:13.807,3248.032867,2024-04-23 15:00:00
4,2024-04-23 16:01:50.036,3225.821195,2024-04-23 16:00:00
...,...,...,...
343,2024-05-07 19:01:18.730,3046.773604,2024-05-07 19:00:00
344,2024-05-07 20:03:20.829,3047.217864,2024-05-07 20:00:00
345,2024-05-07 21:02:42.948,3048.946332,2024-05-07 21:00:00
346,2024-05-07 22:07:34.986,3046.048282,2024-05-07 22:00:00


In [3]:
# ETH donations

# left join on hourly_timestamp
df_eth_donations = pd.merge(df_eth_donations, df_eth_gg20_historical_price, on='hourly_timestamp', how='left')
df_eth_donations = df_eth_donations.rename(columns={'timestamp_x': 'timestamp', 'timestamp_y': 'usd_price_timestamp'})

# add usdValueAtTime
df_eth_donations['usd_value_at_time'] =  df_eth_donations.apply(lambda x: round(Decimal(x['value']) * Decimal(x['usd_price']) / Decimal(1e18), 1), axis=1).astype(float)

df_eth_donations

,hash,from,to,value,timestamp,hourly_timestamp,token,usd_price_timestamp,usd_price,usd_value_at_time
0,0x31623d265c4a96468e8e3f52b1779d39fbb455d125d8...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1000000000000000,2024-05-07 23:39:51,2024-05-07 23:00:00,ETH,2024-05-07 23:04:09.251,3038.919277,3.0
1,0x414efe0670ed36b8026bd9633236e06d3c41247488b8...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,400000000000000,2024-05-07 23:16:05,2024-05-07 23:00:00,ETH,2024-05-07 23:04:09.251,3038.919277,1.2
2,0x4f69f00ce3ef0130d880e9b7c2df7009c47835bc3b36...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,450000000000000,2024-05-07 22:54:26,2024-05-07 22:00:00,ETH,2024-05-07 22:07:34.986,3046.048282,1.4
3,0xc757eebeff0bd14c0d9fb49049794c0fd0f662bae924...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,330000000000000,2024-05-07 22:52:04,2024-05-07 22:00:00,ETH,2024-05-07 22:07:34.986,3046.048282,1.0
4,0xe0c7961e7b9d459ab7d1d4fde33bef6c1a0782ebca6d...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,340000000000000,2024-05-07 22:49:29,2024-05-07 22:00:00,ETH,2024-05-07 22:07:34.986,3046.048282,1.0
...,...,...,...,...,...,...,...,...,...,...
478,0xb7dbe3026ee6cd798eed75446954ad946407d16da76f...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,5310000000000,2024-04-25 03:29:57,2024-04-25 03:00:00,ETH,2024-04-25 03:00:35.783,3153.072544,0.0
479,0xb8cc8ef2cb01e0b9fed6e1ba469c57eddb52d151b76f...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,10000000000000,2024-04-25 03:24:26,2024-04-25 03:00:00,ETH,2024-04-25 03:00:35.783,3153.072544,0.0
480,0x22dd457c866498fd386fe1eec918082e8da495706a8f...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,10000000000000,2024-04-25 03:24:24,2024-04-25 03:00:00,ETH,2024-04-25 03:00:35.783,3153.072544,0.0
481,0xe0173106099473a467b22eb2aeece57557ecb412997c...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,600000000000000,2024-04-24 17:37:28,2024-04-24 17:00:00,ETH,2024-04-24 17:04:25.308,3170.023859,1.9


In [4]:
# ARB donations
# Arbiscan ERC20 Token Txns: https://arbiscan.io/address/0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187#tokentxns

url = (
	"https://api.arbiscan.io/api"
	"?module=account"
	"&action=tokentx"  # action
	f"&address={target_address}"
	f"&contractaddress=0x912CE59144191C1204E64559FE8253a0e49E6548" # ARB
	f"&startblock=0"
	f"&sort=desc"
	f"&apikey={ARBISCAN_API_KEY}"
)

txs = json.loads(requests.get(url).text)['result']

df_arb_donations = get_df_by_txs(txs)
df_arb_donations = filter_gg20_time_range(df_arb_donations)
df_arb_donations['token'] = 'ARB'
df_arb_donations

,hash,from,to,value,timestamp,hourly_timestamp,token
0,0xf6709f4d32440861a46c2dff75cbf323a07efc6fc8ee...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1010000000000000000,2024-05-07 19:26:10,2024-05-07 19:00:00,ARB
1,0x3a02d10bb3f53dd438cf58d3527a69e50195b96464c6...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1000000000000000000,2024-05-07 18:25:12,2024-05-07 18:00:00,ARB
2,0x9630c118e44eb543db9a783ebeb25e1799962e1222c5...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1000000000000000000,2024-05-07 16:45:55,2024-05-07 16:00:00,ARB
3,0x15744d97a17292575b71c34e2d4c16bd79a260b817e3...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1010000000000000000,2024-05-07 15:54:53,2024-05-07 15:00:00,ARB
4,0x1cb02246bad8465fbec6700b227c36d523d163e3bf3a...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1000000000000000000,2024-05-07 15:49:40,2024-05-07 15:00:00,ARB
...,...,...,...,...,...,...,...
66,0xb884bc812f92724d0b0c21346e5fff7d7ed7500792d6...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1000000000000000000,2024-04-25 21:33:27,2024-04-25 21:00:00,ARB
67,0x39ab8dcb97adea6922b805d083f36634561b92622bc6...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1000000000000000000,2024-04-25 21:16:28,2024-04-25 21:00:00,ARB
68,0x9aace009b9d8521533522e8eebc597a6ec94001de19c...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1500000000000000000,2024-04-25 17:04:42,2024-04-25 17:00:00,ARB
69,0x8a8499586ab168369fdabab1dec5c6b3dc86de7ee5ba...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,10000000000000000,2024-04-24 22:59:13,2024-04-24 22:00:00,ARB


In [5]:
# Get arb_gg20_historical_price

with open('gg20/arb_gg20_historical_price.json') as f:
	arb_gg20_historical_price = json.load(f)

# convert to dataframe
df_arb_gg20_historical_price = pd.DataFrame(arb_gg20_historical_price)
df_arb_gg20_historical_price['timestamp'] = pd.to_datetime(df_arb_gg20_historical_price['timestamp'], unit='ms')
df_arb_gg20_historical_price['hourly_timestamp'] = pd.to_datetime(df_arb_gg20_historical_price['timestamp'].apply(lambda x: x.timestamp() / 3600).astype(int), unit='h')

# rename price with usd_price
df_arb_gg20_historical_price = df_arb_gg20_historical_price.rename(columns={'price': 'usd_price'})

df_arb_gg20_historical_price

,timestamp,usd_price,hourly_timestamp
0,2024-04-23 12:02:27.391,1.178624,2024-04-23 12:00:00
1,2024-04-23 13:03:43.991,1.181921,2024-04-23 13:00:00
2,2024-04-23 14:02:37.426,1.190731,2024-04-23 14:00:00
3,2024-04-23 15:03:41.742,1.205438,2024-04-23 15:00:00
4,2024-04-23 16:06:31.981,1.189490,2024-04-23 16:00:00
...,...,...,...
343,2024-05-07 19:05:30.641,1.052451,2024-05-07 19:00:00
344,2024-05-07 20:03:09.541,1.050590,2024-05-07 20:00:00
345,2024-05-07 21:07:38.560,1.051562,2024-05-07 21:00:00
346,2024-05-07 22:08:01.257,1.045444,2024-05-07 22:00:00


In [6]:
# ARB donations
# left join on hourly_timestamp

df_arb_donations = pd.merge(df_arb_donations, df_arb_gg20_historical_price, on='hourly_timestamp', how='left')
df_arb_donations = df_arb_donations.rename(columns={'timestamp_x': 'timestamp', 'timestamp_y': 'usd_price_timestamp'})

# add usdValueAtTime
df_arb_donations['usd_value_at_time'] =  df_arb_donations.apply(lambda x: round(Decimal(x['value']) * Decimal(x['usd_price']) / Decimal(1e18), 1), axis=1).astype(float)

df_arb_donations

,hash,from,to,value,timestamp,hourly_timestamp,token,usd_price_timestamp,usd_price,usd_value_at_time
0,0xf6709f4d32440861a46c2dff75cbf323a07efc6fc8ee...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1010000000000000000,2024-05-07 19:26:10,2024-05-07 19:00:00,ARB,2024-05-07 19:05:30.641,1.052451,1.1
1,0x3a02d10bb3f53dd438cf58d3527a69e50195b96464c6...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1000000000000000000,2024-05-07 18:25:12,2024-05-07 18:00:00,ARB,2024-05-07 18:02:10.226,1.058596,1.1
2,0x9630c118e44eb543db9a783ebeb25e1799962e1222c5...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1000000000000000000,2024-05-07 16:45:55,2024-05-07 16:00:00,ARB,2024-05-07 16:04:40.760,1.063353,1.1
3,0x15744d97a17292575b71c34e2d4c16bd79a260b817e3...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1010000000000000000,2024-05-07 15:54:53,2024-05-07 15:00:00,ARB,2024-05-07 15:07:18.531,1.075304,1.1
4,0x1cb02246bad8465fbec6700b227c36d523d163e3bf3a...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1000000000000000000,2024-05-07 15:49:40,2024-05-07 15:00:00,ARB,2024-05-07 15:07:18.531,1.075304,1.1
...,...,...,...,...,...,...,...,...,...,...
66,0xb884bc812f92724d0b0c21346e5fff7d7ed7500792d6...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1000000000000000000,2024-04-25 21:33:27,2024-04-25 21:00:00,ARB,2024-04-25 21:06:40.835,1.105608,1.1
67,0x39ab8dcb97adea6922b805d083f36634561b92622bc6...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1000000000000000000,2024-04-25 21:16:28,2024-04-25 21:00:00,ARB,2024-04-25 21:06:40.835,1.105608,1.1
68,0x9aace009b9d8521533522e8eebc597a6ec94001de19c...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1500000000000000000,2024-04-25 17:04:42,2024-04-25 17:00:00,ARB,2024-04-25 17:03:28.579,1.101197,1.7
69,0x8a8499586ab168369fdabab1dec5c6b3dc86de7ee5ba...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,10000000000000000,2024-04-24 22:59:13,2024-04-24 22:00:00,ARB,2024-04-24 22:06:41.704,1.125259,0.0


In [7]:
# USDC donations
# Arbiscan ERC20 Token Txns: https://arbiscan.io/address/0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187#tokentxns

url = (
	"https://api.arbiscan.io/api"
	"?module=account"
	"&action=tokentx"  # action
	f"&address={target_address}"
	f"&contractaddress=0xaf88d065e77c8cC2239327C5EDb3A432268e5831" # USDC
	f"&startblock=0"
	f"&sort=desc"
	f"&apikey={ARBISCAN_API_KEY}"
)

txs = json.loads(requests.get(url).text)['result']

df_usdc_donations = get_df_by_txs(txs)
df_usdc_donations = filter_gg20_time_range(df_usdc_donations)
df_usdc_donations['token'] = 'USDC'

# add usd_price_timestamp, usd_price, and usd_value_at_time
df_usdc_donations['usd_price_timestamp'] = None
df_usdc_donations['usd_price'] = 1
df_usdc_donations['usd_value_at_time'] =  df_usdc_donations.apply(lambda x: round(Decimal(x['value']) * Decimal(x['usd_price']) / Decimal(1e6), 1), axis=1).astype(float)

df_usdc_donations

,hash,from,to,value,timestamp,hourly_timestamp,token,usd_price_timestamp,usd_price,usd_value_at_time
0,0x59571d715146ea22c358fa5d28c0e16d079ab2986970...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1080000,2024-05-07 23:24:08,2024-05-07 23:00:00,USDC,None,1,1.1
1,0xe8b879b1170f101d731a372679524788a90348f5adde...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1010000,2024-05-07 22:11:49,2024-05-07 22:00:00,USDC,None,1,1.0
2,0x73f735657846f750d00a2e6a203e396fd81de157eda1...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1000000,2024-05-07 21:48:59,2024-05-07 21:00:00,USDC,None,1,1.0
3,0x16e0cd964135a8a5bc0ed19014549892873a8772a186...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1100000,2024-05-07 21:27:39,2024-05-07 21:00:00,USDC,None,1,1.1
4,0x55b2d893df1b2e087b5b4a96c17c03ff82d5fdd155c3...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1100000,2024-05-07 21:13:33,2024-05-07 21:00:00,USDC,None,1,1.1
...,...,...,...,...,...,...,...,...,...,...
141,0x44e49a65cd7bf0b3446791384181da524d469298c745...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,3200000,2024-04-25 14:15:29,2024-04-25 14:00:00,USDC,None,1,3.2
142,0xfda5580b10bcf79614bb19696d5fd4628c22795982b3...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1010000,2024-04-25 14:09:58,2024-04-25 14:00:00,USDC,None,1,1.0
143,0xec862a8fa5684f4ce39d8d5a634a370ad5325ff0d492...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1000000,2024-04-25 13:59:56,2024-04-25 13:00:00,USDC,None,1,1.0
144,0x94fd4fbd6d488a6dcf07bcdfb6fd3fcfb21df961c2f8...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,110000,2024-04-24 07:45:07,2024-04-24 07:00:00,USDC,None,1,0.1


In [8]:
# Merge three dataframes

df_eth_donations = df_eth_donations.dropna(axis=1, how='all')
df_arb_donations = df_arb_donations.dropna(axis=1, how='all')
df_usdc_donations = df_usdc_donations.dropna(axis=1, how='all')

df_gg20_donations = pd.concat([df_eth_donations, df_arb_donations, df_usdc_donations])
df_gg20_donations = df_gg20_donations.sort_values('timestamp')

df_gg20_donations = df_gg20_donations.reset_index(drop=True)

df_gg20_donations

,hash,from,to,value,timestamp,hourly_timestamp,token,usd_price_timestamp,usd_price,usd_value_at_time
0,0x2e089247ee9aa1545f6da98a254882e30a6801a28b3c...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,50000,2024-04-23 15:46:54,2024-04-23 15:00:00,USDC,NaT,1.000000,0.0
1,0x8eb6a3e3fd9773ab0521285fbff1fa65b541fb2827a1...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,55000000000000,2024-04-23 19:30:27,2024-04-23 19:00:00,ETH,2024-04-23 19:04:20.493,3230.007276,0.2
2,0x477a42de0b89474f91b2de073ae1bdfd79af46cfc1d3...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1000000000000000000,2024-04-23 22:16:29,2024-04-23 22:00:00,ARB,2024-04-23 22:01:47.763,1.176461,1.2
3,0x94fd4fbd6d488a6dcf07bcdfb6fd3fcfb21df961c2f8...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,110000,2024-04-24 07:45:07,2024-04-24 07:00:00,USDC,NaT,1.000000,0.1
4,0xe0173106099473a467b22eb2aeece57557ecb412997c...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,600000000000000,2024-04-24 17:37:28,2024-04-24 17:00:00,ETH,2024-04-24 17:04:25.308,3170.023859,1.9
...,...,...,...,...,...,...,...,...,...,...
695,0xc757eebeff0bd14c0d9fb49049794c0fd0f662bae924...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,330000000000000,2024-05-07 22:52:04,2024-05-07 22:00:00,ETH,2024-05-07 22:07:34.986,3046.048282,1.0
696,0x4f69f00ce3ef0130d880e9b7c2df7009c47835bc3b36...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,450000000000000,2024-05-07 22:54:26,2024-05-07 22:00:00,ETH,2024-05-07 22:07:34.986,3046.048282,1.4
697,0x414efe0670ed36b8026bd9633236e06d3c41247488b8...,0x1B48bB09930676d99dDA36C1aD27ff0a5A5f3911,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,400000000000000,2024-05-07 23:16:05,2024-05-07 23:00:00,ETH,2024-05-07 23:04:09.251,3038.919277,1.2
698,0x59571d715146ea22c358fa5d28c0e16d079ab2986970...,0x8e1bD5Da87C14dd8e08F7ecc2aBf9D1d558ea174,0x9D75F4EbcB8e7669E59dcc27CBadC698E0F77187,1080000,2024-05-07 23:24:08,2024-05-07 23:00:00,USDC,NaT,1.000000,1.1


In [9]:
df_gg20_donations.to_json('vuedapp-gg20-1.json', orient="records")